In [1]:
%matplotlib notebook
import numpy as np
import sigpy as sp
import sigpy.plot as pl
import cupy as cp
import scipy.io
import math
from scipy.sparse import csc_matrix
import numpy.matlib
import matplotlib.pyplot as plt

In [2]:
def wavMask(dims, scale):
    sx, sy = dims
    res = np.ones(dims)
    NM = np.round(np.log2(dims))
    for n in range(int(np.min(NM)-scale+2)//2):
        res[:int(np.round(2**(NM[0]-n))), :int(np.round(2**(NM[1]-n)))] = \
            res[:int(np.round(2**(NM[0]-n))), :int(np.round(2**(NM[1]-n)))]/2
    return res


def imshowWAV(Wim, scale=1):
    plt.imshow(np.abs(Wim)*wavMask(Wim.shape, scale), cmap = plt.get_cmap('gray'))

    
def coeffs2img(LL, coeffs):
    LH, HL, HH = coeffs
    return np.vstack((np.hstack((LL, LH)), np.hstack((HL, HH))))


def unstack_coeffs(Wim):
        L1, L2  = np.hsplit(Wim, 2) 
        LL, HL = np.vsplit(L1, 2)
        LH, HH = np.vsplit(L2, 2)
        return LL, [LH, HL, HH]

    
def img2coeffs(Wim, levels=3):
    LL, c = unstack_coeffs(Wim)
    coeffs = [c]
    for i in range(levels-1):
        LL, c = unstack_coeffs(LL)
        coeffs.insert(0,c)
    coeffs.insert(0, LL)
    return coeffs
    
    
def dwt2(im):
    coeffs = pywt.wavedec2(im, wavelet='db4', mode='per', level=3)
    Wim, rest = coeffs[0], coeffs[1:]
    for levels in rest:
        Wim = coeffs2img(Wim, levels)
    return Wim


def idwt2(Wim):
    coeffs = img2coeffs(Wim, levels=3)
    return pywt.waverec2(coeffs, wavelet='db4', mode='per')

In [3]:
def M_forward(M,c,gpu = False):
    if gpu:
        xp = cp
    else:
        xp = np
    return xp.matmul(M,c)

def C_forward(C,m,gpu = False):
    if gpu:
        xp = cp
    else:
        xp = np
    return xp.matmul(m,C)

def M_adjoint(M,x,gpu = False):
    if gpu:
        xp = cp
    else:
        xp = np
    return xp.matmul(M.T.conj(),x)

def C_adjoint(C,x,gpu = False):
    if gpu:
        xp = cp
    else:
        xp = np
    return xp.matmul(x,C.T.conj())



def soft_thresh_complex(x, l,gpu = False):
    if gpu:
        xp = cp
    else:
        xp = np
    return xp.sign(abs(x)) * xp.maximum(xp.abs(x) - l, 0.)*xp.exp(1j*xp.angle(x))

In [4]:
def R_forward(im,patch_no,patch_size,stride_length):
    [frames,n,m]=im.shape
    n_patch_per_side=math.floor((n-patch_size)/stride_length)+1
    row_patch_no=math.floor((patch_no-1)/n_patch_per_side)
    column_patch_no=(patch_no-1)%n_patch_per_side
    crop=im[:,int(row_patch_no*stride_length):int(row_patch_no*stride_length+patch_size),int(column_patch_no*stride_length):int(column_patch_no*stride_length+patch_size)]
    crop=crop.reshape((frames*patch_size*patch_size,1))
    return crop
def R_adjoint(crop,patch_no,im_size,im_frames,stride_length):
    [a,b]=crop.shape
    patch_size=int((a/im_frames)**(1/2))
#     print(patch_size)
    crop=crop.reshape((im_frames,patch_size,patch_size))
    n_patch_per_side=math.floor((im_size-patch_size)/stride_length)+1
    row_patch_no=math.floor((patch_no-1)/n_patch_per_side)
    column_patch_no=(patch_no-1)%n_patch_per_side
    padded=cp.zeros((im_frames,im_size,im_size),dtype=cp.complex)
    padded[:,int(row_patch_no*stride_length):int(row_patch_no*stride_length+patch_size),int(column_patch_no*stride_length):int(column_patch_no*stride_length+patch_size)]=crop
    return padded


def powermethods(matrix_A,iterations = 10):
    max_eig=0
    for i in range(iterations):
        c = cp.random.rand(32*32,1)
        frwrd=M_forward(matrix_A,c,gpu = True)
        mag_c=cp.dot(cp.transpose(c),c)
        eig=cp.dot(cp.transpose(frwrd),frwrd)/mag_c
        eig = eig[0][0]
#         print(eig)
        max_eig=max(eig,max_eig)
#         print(max_eig)
        
    return float(abs(max_eig))

def modelCforward(M,C):
    # C shape: 1,256,256
    # M shape: 225,24,1032,1032
    patchnum = 25
    result=cp.zeros((24,96,96),dtype=cp.complex)
    for i in range(patchnum):
        M_current=cp.array(M[i,:,:,:].reshape(24*1024,1024))
        patch=R_forward(C,patch_no=i+1,stride_length=16,patch_size=32)
        res = M_forward(M=M_current,c=patch,gpu=True)
#         print(R_adjoint(crop=res,im_frames=24,im_size=256,patch_no=i+1,stride_length=16) )
        result+=R_adjoint(crop=res,im_frames=24,im_size=96,patch_no=i+1,stride_length=16) 
    return result
def modelCadjoint(Im,M):
    # Im shape: (24,256,256)
    # M shape: 225,24,1032,1032
    patch_c = cp.zeros((225,96,96),dtype=cp.complex)
    for i in range(25):
        patch = R_forward(Im,patch_no=i+1,patch_size=32,stride_length=16)
        M_patch = cp.array(M[i,:,:,:].reshape(24*1024,1024))
        c_est = M_adjoint(M_patch,patch,gpu=True)
        radj = R_adjoint(c_est,im_frames=1,patch_no=i+1,stride_length=16,im_size=96)  
        patch_c[i,:,:] = radj.squeeze()
    return patch_c.sum(axis=0)[None,:,:]
def modelMadjoint(Im,C):
    # C shape: (1,256,256)
    # Im shape: (24,256,256)
#     patch_m = np.zeros((225,24,1024,1024),dtype=cp.complex)
    for i in range(25):
        patch = R_forward(Im,patch_no=i+1,patch_size=32,stride_length=16)
        C_patch = R_forward(C,patch_no=i+1,patch_size=32,stride_length=16)
        cadj = C_adjoint(C_patch,patch,gpu=True).reshape(24,1024,1024)
        patch_m[i,:,:,:] = cp.asnumpy(cadj)

In [5]:
kspace = scipy.io.loadmat("cardiac_cine_R6.mat")
sensmaps = kspace["b1"]
raw_data = kspace["kdata"]

In [6]:
def sensforward(LPS_image,sensmaps):
    # LPS (256*24,256)
    # sensmaps (256,256,12)
    LPS = LPS_image.reshape(24,256,256).transpose(1,2,0)
    return LPS[:,:,:,None]*sensmaps[:,:,None,:]
def fftforward(sensmaps):
    # sensmaps (256,256,24,12)
    return sp.fft(sensmaps,axes=(0,1))
def maskforward(fftout,mask):
    # fftout (256,256,24,12)
    # mask (256,256,24,12)
    return fftout*mask
def forwardmodel(LPS_image,sensmaps,mask):
    return maskforward(fftforward(sensforward(LPS_image,sensmaps)),mask)
def maskadjoint(kspace,mask):
    return kspace*mask
def fftadjoint(maskout):
    return sp.ifft(maskout,axes=(0,1))
def sensadjoint(fftout,sensmaps):
    ssp = sensmaps[:,:,None,:].conj()
    return cp.sum(fftout*ssp,axis=3)
def adjointmodel(kspace,sensmaps,mask):
    return sensadjoint(fftadjoint(maskadjoint(kspace,mask)),sensmaps)

In [7]:
def soft_thresh_complex(x, l):
    return cp.sign(abs(x)) * np.maximum(cp.abs(x) - l, 0.)*cp.exp(1j*cp.angle(x))
def ista_kspace(C1,M1,L,lamda,CS,x,N,sens,mask):
    C0=C1
    M0=M1
    converge = []
    for i in range(N):
#         L = float(cp.linalg.eigvalsh(M0.T.dot(M0))[-1])
        for j in range(100):
#             print(j)
            C01 = C0 - (1/L)*M_adjoint(M0,adjointmodel(forwardmodel(M_forward(M0,C0),sens,mask)-x,sens,mask).transpose(2,0,1).reshape(24*256,256))
            w = dwt2(cp.asnumpy(C01))
            C00 = soft_thresh_complex_np(w,CS/L)
            C01 = cp.asarray(idwt2(C00))
            converge.append(cp.linalg.norm(forwardmodel(M_forward(M0,C0),sens,mask)-x))
            C0 = C01
#         L = float(cp.linalg.eigvalsh(C0.T.dot(C0))[-1])
        for t in range(100):
            w = M0 - (1/L)*C_adjoint(C0,adjointmodel(forwardmodel(C_forward(C0,M0),sens,mask)-x,sens,mask).transpose(2,0,1).reshape(24*256,256))
            M01 = soft_thresh_complex(w,lamda/L)
            converge.append(cp.linalg.norm(forwardmodel(C_forward(C0,M0),sens,mask)-x))
            M0 = M01
        print(cp.linalg.norm(forwardmodel(C_forward(C0,M0),sens,mask)-x))
        cp.save("M1.npy",M0)
        cp.save("C1.npy",C0)
        cp.save("converge1.npy",converge)
        
    return M0,C0,converge

def fista_kspace(C1,M1,L,lamda,CS,x,N,sens,mask):
    C00=C1
    M00=M1
    converge = []
    for i in range(N):
#         L = float(cp.linalg.eigvalsh(M0.T.dot(M0))[-1])
        P0 = C00
        tk = 1
        for j in range(100):
#             print(j)
            W00 = P0 - (1/L)*M_adjoint(M00,adjointmodel(forwardmodel(M_forward(M00,P0),sens,mask)-x,sens,mask).transpose(2,0,1).reshape(24*256,256))
            w = dwt2(cp.asnumpy(W00))
            W01 = soft_thresh_complex_np(w,CS/L)
            C01 = cp.asarray(idwt2(W01))
            tk1 = (1+np.sqrt(1+4*tk*tk))/2
            bk = (tk-1)/tk1
            P0 = C01+bk*(C01-C00)
            converge.append(cp.linalg.norm(forwardmodel(M_forward(M00,C00),sens,mask)-x))
            C00 = C01
            tk = tk1
        Q0 = M00
        vk = 1
#         L = float(cp.linalg.eigvalsh(C0.T.dot(C0))[-1])
        for t in range(100):
            wk = Q0 - (1/L)*C_adjoint(C00,adjointmodel(forwardmodel(C_forward(C00,Q0),sens,mask)-x,sens,mask).transpose(2,0,1).reshape(24*256,256))
            M01 = soft_thresh_complex(wk,lamda/L)
            vk1 = (1+np.sqrt(1+4*vk*vk))/2
            jk = (vk-1)/vk1
            Q0 = M01+jk*(M01-M00)
            vk = vk1
            converge.append(cp.linalg.norm(forwardmodel(C_forward(C00,M00),sens,mask)-x))
            M00 = M01
        print(cp.linalg.norm(forwardmodel(C_forward(C00,M00),sens,mask)-x))
        cp.save("M11.npy",M00)
        cp.save("C11.npy",C00)
        cp.save("converge1.npy",converge)
        
    return M00,C00,converge